In [2]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import os
import matplotlib.image as mpimg
from pathlib import Path
import re
import imageio 

In [3]:
# Paths:
main_path = '/home/avoyeux/old_project/avoyeux'
stereo_img_path = os.path.join(main_path, '..', 'backup', 'int')
avg_stereo_img_path = os.path.join(main_path, 'ratio')
stereo_mask_path = os.path.join(main_path, 'prog', 'masque')

plot_path = os.path.join(main_path, 'Rainbow_plots')
os.makedirs(plot_path, exist_ok=True)

In [4]:
# Getting the image and masks path names (to then correctly match them together)
image_names = sorted(Path(stereo_img_path).glob('*.png'))
avg_names = sorted(Path(avg_stereo_img_path).glob('*.png'))
mask_names = sorted(Path(stereo_mask_path).glob('*.png'))

# Getting the corresponding image and mask numbers 
mask_pattern = re.compile(r'frame(\d{4})\.png')
mask_numbers = [int(mask_pattern.match(os.path.basename(mask_name)).group(1)) for mask_name in mask_names]

# as ratio has all the numbers, image_numbers is useless

In [5]:
# Code to get the contours of a mask
def Contours(mask):
    """Function to plot the contours given a mask
    Source: https://stackoverflow.com/questions/40892203/can-matplotlib-contours-match-pixel-edges"""

    pad = np.pad(mask, [(1, 1), (1, 1)])  # zero padding
    im0 = np.abs(np.diff(pad, n=1, axis=0))[:, 1:]
    im1 = np.abs(np.diff(pad, n=1, axis=1))[1:, :]
    lines = []
    for ii, jj in np.ndindex(im0.shape):
        if im0[ii, jj] == 1:
            lines += [([ii - .5, ii - .5], [jj - .5, jj + .5])]
        if im1[ii, jj] == 1:
            lines += [([ii - .5, ii + .5], [jj - .5, jj - .5])]

    return lines

In [6]:
def Grid_line_positions(cen, width, image_shape, axis, dx=0.075, deg_grid_width=15):

    # Image border
    border = cen - width / 2

    # Getting the first grid line position
    first_pos = border
    for loop in range(image_shape[axis]):
        if round(first_pos, 2) % deg_grid_width == 0:
            img_index = loop
            break
        first_pos += dx

    # Getting the rest of the gridline positions
    positions = np.arange(img_index, image_shape[axis] + 0.1, deg_grid_width/dx, dtype='uint16')
    values = np.arange(round(first_pos, 2), 
                       first_pos + (len(positions) - 1) * deg_grid_width + 1e-4, deg_grid_width)

    text_val = []
    for value in values:
        if axis == 1:
            if value > 180:
                value -= 360
                text = f'{abs(value)}° W'
            else:
                text = f'{value}° E'
            text_val.append(text)
        else:
            if value > 0:
                text = f'{value}° N'
            else:
                text = f'{abs(value)}° S'
            text_val.append(text)
    return positions, text_val

In [7]:
# Adding the grid lines and the corresponding text 
def Grid_linesntext(ax, lat_lines, lat_text, lon_lines, lon_text):
    linewidth = 0.4
    alpha = 0.4
    color = 'white'
    size = 3
    linestyle = '--'
    for pos, lat_line in enumerate(lat_lines):
        ax.axhline(lat_line, color=color, linestyle=linestyle, linewidth=linewidth, alpha=alpha)
        ax.text(3, lat_line - 2, lat_text[-(1 + pos)], color=color, alpha=alpha, size=size)
    for pos, lon_line in enumerate(lon_lines):
        ax.axvline(lon_line, color=color, linestyle=linestyle, linewidth=linewidth, alpha=alpha)
        ax.text(lon_line, 10, lon_text[pos], color=color, alpha=alpha, size=size)

In [8]:
# Plot saving function
def Plotting_func(number, stereo_image, avg_image, lines, loop, fig_stats):
    loncen, latcen, lonwidth, latwidth, dlon, dlat = fig_stats
    # Creating the grid
    lon_lines, lon_text = Grid_line_positions(loncen, lonwidth, avg_image.shape, 1)
    lat_lines, lat_text = Grid_line_positions(latcen, latwidth, avg_image.shape, 0)

    # Plotting both images right next to each other 
    fig, axs = plt.subplots(1, 3, figsize=(8, 8))

    # For the first image
    axs[0].imshow(stereo_image, interpolation='none')
    Grid_linesntext(axs[0], lat_lines, lat_text, lon_lines, lon_text)
    axs[0].axis('off')
    axs[0].set_title(f'{os.path.basename(image_names[number])}')

    # The contrast image 
    axs[1].imshow(avg_image, interpolation='none')
    Grid_linesntext(axs[1], lat_lines, lat_text, lon_lines, lon_text)
    axs[1].axis('off')
    axs[1].set_title(f'img{os.path.basename(avg_names[number])}, mask{os.path.basename(mask_names[loop])}')
    plt.tight_layout()

    # For the contrast with the mask lines
    axs[2].imshow(avg_image, interpolation='none')
    Grid_linesntext(axs[2], lat_lines, lat_text, lon_lines, lon_text)
    for line in lines:
        axs[2].plot(line[1], line[0], color='r', linewidth=0.5, alpha=0.3)
    axs[2].axis('off')
    axs[2].set_title(f'img{os.path.basename(avg_names[number])}, mask{os.path.basename(mask_names[loop])}')
    plt.tight_layout()

    fig_name = f'Plot_{number:04d}.png'
    plt.savefig(os.path.join(plot_path, fig_name), bbox_inches='tight', pad_inches=0.05, dpi=800)
    plt.close()

In [8]:
# Stats for the images
loncen = 195
latcen = 0
lonwidth = 45
latwidth = 45
dlon = 0.075
dlat = 0.075
merlonmax = 65 # no clue what this is 

img_stats = loncen, latcen, lonwidth, latwidth, dlon, dlat

# Adding the mask to the corresponding image
for loop, number in enumerate(mask_numbers):
    # Uploads and initialisation
    stereo_image = mpimg.imread(image_names[number])
    avg_image = mpimg.imread(avg_names[number])
    rgb_mask = mpimg.imread(mask_names[loop])

    # Changing to gray_scale and getting the mask contours
    gray_mask = np.mean(rgb_mask, axis=2)
    normalised_mask = 1 - gray_mask 
    normalised_mask /= np.max(normalised_mask) 
    filters = (normalised_mask == 0)
    normalised_mask[filters] = np.nan # to be used with the 'Reds' cmap

    # Creating a bool array to get the contours 
    bool_array = ~np.isnan(normalised_mask)
    lines = Contours(bool_array)

    Plotting_func(number, stereo_image, avg_image, lines, loop, img_stats)
    print(f'Plotting for image nb {number} done.')

values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0.  15.]
values are [180. 195. 210.]
values are [-15.   0

In [10]:
# Creating a gif with the saved plots
fps = 0.8
img_paths = [os.path.join(plot_path, f'Plot_{number:04d}.png') for number in mask_numbers]
with imageio.get_writer(os.path.join(plot_path, 'the_gif.gif'), mode='I', duration=fps*1000) as writer:
    for img_path in img_paths:
        image = imageio.imread(img_path)
        writer.append_data(image)

/tmp/ipykernel_2330491/679047245.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(img_path)


## For testing

In [ ]:
stereo_image = mpimg.imread(image_names[85])
avg_image = mpimg.imread(avg_names[85])
rgb_mask = mpimg.imread(mask_names[0])

gray_mask = np.mean(rgb_mask, axis=2)
normalised_mask = 1 - gray_mask 
normalised_mask /= np.max(normalised_mask) 

filters = (normalised_mask == 0)
normalised_mask[filters] = np.nan # to be used with the 'Reds' cmap

# Creating a bool array to get the contours 
bool_array = ~np.isnan(normalised_mask)
lines = Contours(bool_array)